Import modules to load and process the data. Numpy handels data chrunching, h5py is responsible for file I/O

In [2]:
import numpy as np
import h5py
import os

Define the rawdata path and optional the desired acceleration

In [28]:
    path = 'rawdata_brain_radial_96proj_12ch.h5'
    acc = 1    

Read in Data. The if/else for the heart is used to select the exact number of spokes as in the reference paper.

In [12]:
    if path == '':
        raise ValueError("No data file specified")

    name = os.path.normpath(path)
    h5_dataset = h5py.File(name, 'r')
    h5_dataset_rawdata_name = 'rawdata'
    h5_dataset_trajectory_name = 'trajectory'

    if "heart" in name:
        if acc == 2:
            trajectory = h5_dataset.get(h5_dataset_trajectory_name)[
                :, :, :33]
            rawdata = h5_dataset.get(h5_dataset_rawdata_name)[
                :, :, :33, :]
        elif acc == 3:
            trajectory = h5_dataset.get(h5_dataset_trajectory_name)[
                :, :, :22]
            rawdata = h5_dataset.get(h5_dataset_rawdata_name)[
                :, :, :22, :]
        elif acc == 4:
            trajectory = h5_dataset.get(h5_dataset_trajectory_name)[
                :, :, :11]
            rawdata = h5_dataset.get(h5_dataset_rawdata_name)[
                :, :, :11, :]
        else:
            trajectory = h5_dataset.get(h5_dataset_trajectory_name)[...]
            rawdata = h5_dataset.get(h5_dataset_rawdata_name)[...]
    else:
        trajectory = h5_dataset.get(h5_dataset_trajectory_name)[
            :, :, ::acc]
        rawdata = h5_dataset.get(h5_dataset_rawdata_name)[
            :, :, ::acc, :]

    # Squeeze dummy dimension and transpose to C-style ordering.
    rawdata = np.squeeze(rawdata.T)

    # Norm Trajectory to the range of (-1/2)/(1/2)
    trajectory = np.require((
        trajectory[0]/(2*np.max(trajectory[0])) +
        1j*trajectory[1]/(2*np.max(trajectory[0]))).T,
                   requirements='C')

    # Close file after everything was read
    h5_dataset.close()

Setup dict to hold essential data properties. The over grid factor (ogf) defines the amount of oversampling of the non Cartesian k-space gridded on an Cartesian grid. The brain data is oversampled by 1.706, the heart by a factor of 1+1/3.

In [29]:
    # Create empty dict
    par = {}
    [nCh, nSpokes, nFE] = rawdata.shape

    par["ogf"] = 1.706
    dimX, dimY = [int(nFE/par["ogf"]), int(nFE/par["ogf"])]
    
    par["NC"] = nCh
    par["dimY"] = dimY
    par["dimX"] = dimX
    par["nFE"] = nFE
    par["Nproj"] = nSpokes

Calculate density compensation function (dcf). Currently assumes golden angle radial trajectories

In [24]:
    if len(np.shape(trajectory)) == 2:
        nspokes, N = np.shape(trajectory)
    elif len(np.shape(trajectory)) == 3:
        NScan, nspokes, N = np.shape(trajectory)
    else:
        raise ValueError("Passed trajectory has the wrong "
                         "number of dumensions.")

    dcf = np.abs(np.linspace(-N/2, N/2, N))/(N/2)  # ramp from 1...1
    dcf = np.repeat(dcf, nspokes, 0)
    dcf /= np.min(dcf)
    dcf *= (np.pi / 4) / nspokes
    dcf = np.reshape(dcf, (N, nspokes)).T

Apply square root of dcf to data to ensure adjointness of nuFFT operations

In [30]:
dcf = np.sqrt(dcf)
rawdata /= dcf

Perform the Iterative Reconstruction

RECON CODE GOES HERE

Save the results

In [31]:
    outdir = ""
    if "heart" in path:
        outdir += "/heart"
    elif "brain" in path:
        outdir += "/brain"
    if not os.path.exists('./output'):
        os.makedirs('output')
    if not os.path.exists('./output' + outdir):
        os.makedirs("./output" + outdir)
    cwd = os.getcwd()
    os.chdir("./output" + outdir)
    f = h5py.File("CG_reco_inscale_" +
                  "_acc_" + str(acc), "w")
    f.create_dataset("CG_reco", result.shape,
                     dtype=DTYPE, data=result)
    f.flush()
    f.close()
    os.chdir(cwd)


NameError: name 'result' is not defined